## 📚 שלב 1: ייבוא הספריות

**הספריות שנשתמש בהן:**
- `cv2` (OpenCV) - ספרייה לעבודה עם מצלמות ותמונות
- `Flask` - שרת אינטרנט קטן שישדר את התמונות לטלפון
- `threading` - כדי להריץ את השרת ברקע

**מושגים בסיסיים:**
- **Frame** = תמונה אחת מתוך הוידאו (כמו דף בספר מצויר)
- **FPS** = כמה תמונות בשנייה (30 FPS = 30 תמונות בשנייה = וידאו חלק)
- **Resolution** = גודל התמונה (640x480 = רוחב 640 פיקסלים, גובה 480)

In [1]:
import os
os.environ["QT_QPA_PLATFORM"] = "xcb"  # הגדרה טכנית ל-Raspberry Pi

import cv2
import time
import threading
from flask import Flask, Response

print("✅ כל הספריות נטענו בהצלחה!")

✅ כל הספריות נטענו בהצלחה!


---

## 🎥 שלב 2: פתיחת המצלמה (פשוט)

נתחיל עם הדרך הכי פשוטה - לפתוח מצלמה ולהציג תמונות חיות על המסך.

**פרמטרים חשובים:**
- `VideoCapture(0)` - המספר 0 אומר "המצלמה הראשונה"
- `CAP_PROP_FRAME_WIDTH` - רוחב התמונה
- `CAP_PROP_FRAME_HEIGHT` - גובה התמונה
- `CAP_PROP_FPS` - כמה תמונות בשנייה

**איך זה עובד:**
1. פותחים את המצלמה
2. בלולאה: קוראים תמונה אחת
3. מציגים אותה על המסך
4. חוזרים לשלב 2 (עד שלוחצים 'q' לעצירה)

In [3]:
# Best-practice smooth live camera in Jupyter: ipywidgets.Image (no matplotlib redraw)
# Stop the stream by interrupting the cell (Kernel -> Interrupt) or the stop button.

import cv2
import time
import ipywidgets as widgets
from IPython.display import display

# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20                 # widgets can usually handle 15-25 fps smoothly
JPEG_QUALITY = 75               # 60-85 is a good range
DROP_GRABS = 2                  # drop buffered frames to reduce lag
DT = 1.0 / TARGET_FPS

try:
    t_next = time.time()
    while True:
        # drop a couple frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        # encode BGR -> JPEG (fast + smooth in notebooks)
        ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output (prevents flicker + CPU spikes)
        now = time.time()
        sleep = t_next - now
        if sleep > 0:
            time.sleep(sleep)
        t_next = max(t_next + DT, now)

finally:
    cap.release()


Image(value=b'', format='jpeg')

KeyboardInterrupt: 

---

## 📱 שלב 3: שידור לטלפון (Streaming)

עכשיו נוסיף שרת אינטרנט קטן שישדר את התמונות לטלפון.

**איך זה עובד בקצרה:**
1. השרת Flask רץ ברקע
2. הוא "מאזין" לכתובת `http://10.2.0.17:8080`
3. כשנפתח את הכתובת בטלפון, השרת שולח תמונות חדשות כל הזמן
4. זה נקרא **MJPEG stream** - כמו "תמונות רצות"

**מה קורה פה:**
- `@app.route("/")` = דף הבית (עם כפתורים)
- `@app.route("/video_feed")` = הוידאו עצמו
- `generate_stream()` = פונקציה שמייצרת תמונות אחת אחרי השנייה

**לא צריך להבין את כל הקוד!** זה רק "מתכון" שמשדר תמונות.

In [ ]:
# test_wifi script

---

## 🔧 ניסוי: שינוי רזולוציה ו-FPS

נסו לשנות את ההגדרות ולראות איך זה משפיע:

| פרמטר | ערך נמוך | ערך בינוני | ערך גבוה | השפעה |
|--------|----------|------------|----------|--------|
| **Width** | 320 | 640 | 1280 | רוחב התמונה - גדול יותר = איכות טובה יותר אבל איטי יותר |
| **Height** | 240 | 480 | 720 | גובה התמונה |
| **FPS** | 15 | 30 | 60 | תמונות בשנייה - גבוה יותר = חלק יותר אבל עוזר יותר CPU |

**דוגמה לרזולוציה נמוכה (מהיר):**

In [ ]:
# ניסוי: רזולוציה נמוכה
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)   # רוחב קטן
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)  # גובה קטן
cap.set(cv2.CAP_PROP_FPS, 15)            # 15 תמונות לשנייה

print("📷 מצלמה ברזולוציה נמוכה (320x240, 15 FPS)")
print("זה יהיה מהיר אבל פחות חד")

for i in range(50):
    ret, frame = cap.read()
    if ret:
        cv2.imshow("רזולוציה נמוכה", frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
print("✅ סיימתי")

---

## 📊 סיכום - פקודות ופרמטרים

### פקודות מצלמה:

| פקודה | תיאור | פרמטרים |
|-------|--------|----------|
| `cv2.VideoCapture(0)` | פותח מצלמה | 0 = מצלמה ראשונה |
| `cap.set(property, value)` | שינוי הגדרות | ראו טבלה למטה |
| `cap.read()` | קריאת תמונה | מחזיר: (הצלחה?, תמונה) |
| `cv2.imshow(name, img)` | הצגת תמונה | שם החלון, התמונה |
| `cv2.waitKey(1)` | המתנה 1ms + בדיקת מקש | מחזיר: קוד המקש |
| `cap.release()` | סגירת המצלמה | - |
| `cv2.destroyAllWindows()` | סגירת כל החלונות | - |

### הגדרות מצלמה (properties):

| Property | ערך | תיאור |
|----------|-----|--------|
| `CAP_PROP_FRAME_WIDTH` | 320-1920 | רוחב פיקסלים |
| `CAP_PROP_FRAME_HEIGHT` | 240-1080 | גובה פיקסלים |
| `CAP_PROP_FPS` | 15-60 | תמונות לשנייה |

---

## 💡 רעיונות לניסויים

1. **הוספת טקסט על התמונה:**
   ```python
   cv2.putText(frame, "שלום!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
   ```

2. **שמירת תמונה:**
   ```python
   cv2.imwrite("תמונה.jpg", frame)
   ```

3. **המרה לשחור-לבן:**
   ```python
   gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   cv2.imshow("שחור לבן", gray)
   ```

4. **ספירת פריימים:**
   ```python
   count = 0
   # בתוך הלולאה:
   count += 1
   cv2.putText(frame, f"Frame: {count}", (10, 30), ...)
   ```

---

## ⚠️ טיפולים בבעיות נפוצות

| בעיה | פתרון |
|------|-------|
| "לא הצלחתי לפתוח מצלמה" | בדקו שהמצלמה מחוברת. נסו `ls /dev/video*` בטרמינל |
| "התמונה קפואה" | המצלמה תקועה - הריצו `sudo systemctl restart nvargus-daemon` |
| "השרת לא עובד בטלפון" | בדקו שאתם באותה רשת Wi-Fi. נסו `ifconfig` לראות את ה-IP |
| "התמונה שחורה" | יתכן שהמצלמה מכוסה או שיש בעיה ברשיונות |

**בהצלחה! 📸**